# JC 模型
最简单的光与原子相互作用的模型：单模腔中的二能级原子（偶极相互作用）


## 哈密顿量
$$
H = \hbar \omega_c a^\dagger a + \frac{1}{2}\hbar \omega_a \sigma_z + \hbar g (a + a^\dagger)(\sigma_- + \sigma_+)
$$

## 旋波近似下（RWA）
忽略掉高频震荡
哈密顿量为
$$
H = \hbar \omega_c a^\dagger a + \frac{1}{2}\hbar \omega_a \sigma_z + \hbar g (a\sigma_+ + a^\dagger \sigma_-)
$$

### 这里，我们令 $\hbar = 1$

In [1]:
# 导入必要的包
import matplotlib.pyplot as plt
import numpy as np 
from qutip import *

In [2]:
# 定义相关常数
omega_c = 1.0 * 2 * np.pi # 腔频
omega_a = 1.0 * 2 * np.pi # 原子跃迁频率
g = 0.05 * 2 * np.pi # 耦合强度
kappa = 0.005 # 腔耗散率
gamma = 0.05 # 原子耗散率
N = 15 # 腔最大光子数（在15处截断）
n_th_a = 0 # 热激发平均光子数
use_rwa = True # 取旋波近似

# 演化时间
tlist = np.linspace(0, 25, 101)